# Oasis Montaj control file generator
_Because I hate clicking_

## Workflow
### Setup - Import and QC your line database.
1. Download a line survey from GADDS, etc, in EPSG:4283 (GDA94) _You may use GDA20 as appropriate_
1. Import (ASCII) a line dataset into an Oasis Montaj database, using your template (see e.g. ./GA_ASCII_import.i3) _You may need to increase the database size to 12, etc._
1. Database Tools > Split on Line Channel, _line_
1. Coordinates > Set Coordinate System, _GDA94_
1. Coordinates > New Projected Coordinate System, e.g. from _GDA94_ to  _GDA94 / MGA Zone 55_
1. Coordinates > Set current X,Y,Z... to _long\_mga55_, etc...
1. _You may now wish to inspect the _Line Path_ in a _New Map_. We will be modifying these line data._
    a. Specifically, we will be dropping tie lines, and decimating every nth line.
1. _You may now wish to visualise a low resolution grid to check the import. Grid and Image > Grid Data_
1. Export a copy of the __lineType__ _Database Tools > Reports > Line/Channel report_, which will be used for the _stats_file_name_ below.


### Execution - Execute code, decimate lines, and grid
1. Run the below cells, specifying your created stats.txt, survey name, parameters, etc. Check control file looks acceptable
1. Create 5 copies of the database, named Pxxx_1, Pxxx_2, Pxxx_3, Pxxx_4, Pxxx_all. The original database serves as a backup copy. _These will be decimated accordingly._
1. With the corresponding database selected:
    1. Delete tie lines using the selection file or the Oasis Montaj _Select/Deselect Type_ feature
    1. Select the lines to be deleted using _Get Saved Selections_ with the selection file (will incorporate Tie lines if not already deleted). _n.b. this *deselects* lines that will be *kept* for gridding_
    1. Delete the lines, and select all remaining lines
    1. _Grid and Image > Gridding > From control file > Minimum Curvature Control File_, select your data channel and file names as appropriate .ers (copy control file path from output below)





In [ ]:
import logging
import re
from pathlib import Path
import numpy as np

logging.basicConfig()

survey_name = "P739"
om_working_dir = "C:/Luke/PhD/oasis montaj/ArbSR"
nominal_line_spacing = 250  # m
decimation_factors = [1, 2, 3, 4]
# xmin, ymin, xmax, ymax = (356800,6124400,504350,6350900,)  # W, S, E, N / xmin, ymin, xmax, ymax


def create_sel_file(
    in_dir="",
    stats_file_name="stats.txt",
    out_prefix="decimated",
    data_line_type=2,
    tie_line_type=4,
    decimation_factors=[1],
    survey_name="",
):
    """Prepare a selection file for Oasis Montaj, based on all available lines

    By default, only tie lines "type 4" are dropped - specify a list of
    decimation factors other than the default in order to remove lines.

    After using the selection tool on your line database, run dellines.gx on
    selected lines.

    LSgeo 2021
    """

    line_meta = Path(in_dir) / stats_file_name
    if not line_meta.exists():
        raise FileNotFoundError("Export lineType channel report from OM first!")

    ch_sec_summ = line_meta.read_text().split("ALL")[1].split("\n")[0]
    exponent_num = re.compile(r"([0-9.+e\-]+)\s+")
    xmin, xmax, ymin, ymax, _ = np.array(
        re.findall(exponent_num, ch_sec_summ), dtype=float
    )
    for i in [[xmin, "xmin"], [xmax, "xmax"], [ymin, "ymin"], [ymax, "ymax"]]:
        if abs(i[0]) > 1e10:
            logging.error(
                f"Value of {i[0]} is out of range for {i[1]}, determine manually"
            )

    # Get the min, max, average section of statistics report
    ch_sec = line_meta.read_text().split(
        "------------   -------        -------        ----           --------       ----------- ------------"
    )[-1]

    all_lines = []
    for line in ch_sec.split("\n"):  # Split by line
        if ":" in line:  # Only care about line number rows, not blank or header
            for i in line.split(","):
                line_name = i.split("      ")[0]
                line_type = i.split("              ")[2]
                all_lines.append([f'"{line_name[:-2]}"', int(line_type)])
                if "." in line_type:
                    raise NotImplementedError(
                        "Line probably contains mixed types, mad sorry bro"
                    )

    for df in decimation_factors:
        decimation_counter = 0
        # Save every n lines
        selection_to_delete = []
        logging.info(f"\n\nRunning Decimation Factor {df}")

        for line in all_lines:
            if line[1] == tie_line_type:
                selection_to_delete.append(line[0])
                logging.info(f"Will delete tie Line {line[0]}")
                continue  # Handle tie lines independent of decimation

            if decimation_counter % df != 0:
                selection_to_delete.append(line[0])
                logging.info(f"Will delete Line {decimation_counter}: {line[0]}")

            decimation_counter += 1  # Start at line 0 with this at end of loop

        sel_file = Path(in_dir) / f"{survey_name}_{out_prefix}_{df}.sel"
        sel_file.write_text("\n".join(selection_to_delete))

        print(sel_file.absolute())

    return xmin, xmax, ymin, ymax


xmin, xmax, ymin, ymax = create_sel_file(
    in_dir=om_working_dir,
    stats_file_name=f"{survey_name}_stats.txt",
    decimation_factors=decimation_factors,
    survey_name=survey_name,
)
xmax = 13_710_000
ymin = 213_000



In [ ]:
print(xmin)
print(xmax)
print(ymin)
print(ymax)



In [ ]:
def create_grid_control_file(
    in_dir="",
    out_prefix="decimated",
    decimation_factor=1,
    hr_line_spacing=None,
    along_line_spacing=None,
    cs_fac=5,  # Default line spacing:cell size ratio
    cs=None,
    xmin="",
    ymin="",
    xmax="",
    ymax="",
    bclip="",  # n.b. included in rangrid.con, but not OM help.
    chan="",
    zb="",
    zm="",
    logopt="",
    logmin="",
    idsf="",
    bkd="",
    srd="",
    iwt="",
    edgclp="",
    wtslp="",  # n.b. not included in example rangrid.con, but listed in OM help.
    tol="",
    pastol="",
    itrmax="",
    ti="",
    icgr="",
):

    """Create control file for Minimum Curvature Gridding in Oasis Montaj

    Priority is given to specified args, then python defaults, then OM defaults
    See rangrid.con example in this folder for explanation and OM defaults.
    """

    line_spacing = hr_line_spacing * decimation_factor
    cs = cs or line_spacing / cs_fac  # between 1/5 and 1/2 is good
    bkd = bkd if bkd != "" else 10 * cs
    wtslp = wtslp if wtslp != "" else line_spacing / cs
    # If the data has been collected along traverse lines, it is recommended to set the slope (wtslp) as the number of grid points between the traverse lines

    if cs % 1 != 0:
        logging.info(
            f"Decimation Factor {decimation_factor} will result in non-decimal cell size. This is supported, but may be undesired."
        )

    con_str = (
        f"{cs}/\n"
        f"{xmin},{ymin},{xmax},{ymax},{bclip}/\n"
        f"{chan},{zb},{zm},{logopt},{logmin}/\n"
        f"{idsf},{bkd},{srd},{iwt},{edgclp},{wtslp}/\n"
        f"{tol},{pastol},{itrmax},{ti},{icgr}/\n"
    )
    out_file = (
        Path(in_dir)
        / f"{survey_name}_{out_prefix}_{decimation_factor}_{chan}_{cs:2.1f}_rangrid.con"
    )
    out_file.write_text(con_str)

    print(out_file.absolute().as_posix())


for df in decimation_factors:
    create_grid_control_file(
        in_dir=om_working_dir,
        xmin=xmin,
        ymin=ymin,
        xmax=xmax,
        ymax=ymax,
        chan="mag_awagsLevelled",
        decimation_factor=df,
        hr_line_spacing=nominal_line_spacing,
        cs_fac=4,
        tol=1,  # Data range ~ -500, 500
        ti=0,
        itrmax=200,
    )


In [ ]:
# TODO ers to tiled tifs.
